In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel
import torch
from gradio_client import Client, handle_file
from IPython.display import Audio 


In [13]:
folder = "./final_model/"
base_model = BertForSequenceClassification.from_pretrained(folder, num_labels=7)
peft_model = PeftModel.from_pretrained(base_model, folder)
tokenizer = BertTokenizer.from_pretrained(folder)
peft_model.eval()

Some weights of the model checkpoint at ./final_model/ were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.9.attention.self.value.lora_A.default.weight', 'bert.encoder.layer.3.output.dense.lora_A.default.weight', 'bert.encoder.layer.10.intermediate.dense.lora_B.default.weight', 'bert.encoder.layer.3.intermediate.dense.lora_B.default.weight', 'bert.encoder.layer.10.output.dense.lora_B.default.weight', 'bert.encoder.layer.8.attention.self.value.lora_A.default.weight', 'bert.encoder.layer.1.attention.self.key.lora_A.default.weight', 'bert.encoder.layer.4.intermediate.dense.lora_B.default.weight', 'bert.encoder.layer.7.output.dense.lora_A.default.weight', 'bert.encoder.layer.8.attention.output.dense.lora_B.default.weight', 'bert.encoder.layer.10.output.dense.lora_A.default.weight', 'bert.encoder.layer.11.attention.self.query.lora_B.default.weight', 'bert.encoder.layer.6.attention.self.key.lora_B.default.weight', 'bert.encoder.layer.3.attention.self.key.lora_

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(100000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.2, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out

In [14]:
names={'Egyptian':0, 'Iraqi':1, 'Khaleeji':2, 'Levantine':3, 'Lybia':4, 'Maghrebi':5, 'Sudani':6}

In [15]:
import os
def voice(text):
    try:
        client = Client("MohamedRashad/Egyptian-Arabic-TTS")
        result = client.predict(
            text=text,
            speaker_audio_path=handle_file('https://github.com/gradio-app/gradio/raw/main/test/test_files/audio_sample.wav'),
            temperature=1,
            api_name="/infer_EGTTS"
        )
        with open(result, "rb") as audio_file:
            audio_data = audio_file.read()
        os.remove(result)
        return audio_data
    except Exception as e:
        print(f"TTS error: {e}")
        return b""  # Return empty bytes to avoid crash

In [28]:
input_text = """
السفر بيخليني أحس إني حر، بعيد عن الروتين والضغط والزحمة، حتى لو رايح مكان بسيط، بس التغيير في حد ذاته بيعمل فرق، بتشوف ناس جديدة، مناظر مختلفة، وتفكر في حاجات مكانتش تخطر على بالك، كأنك بتعمل إعادة تشغيل لنفسك كده.
"""

In [32]:
input_text = """
الواحد مطلوب منه انه يكتب كلام عشان يبين ان النوذج الذكاء الاصطناعي ده شغال.
 بس عرفت افكر في فكرة حلوة اني اكتب عن اني زهقان وده خلاني اعرف اكتب شوية كلام مش قليل يعني علي قد المطلوب.
"""

In [33]:
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,
    max_length=512,
    truncation=True,
)
inputs = inputs.to(peft_model.device)
with torch.no_grad():
    outputs = peft_model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
names2 = {v: k for k, v in names.items()}
predicted_label = names2[predictions.item()]
print(f"Predicted dialect: {predicted_label}")
audio_data = voice(input_text)
Audio(audio_data, autoplay=True, rate=24000)

Predicted dialect: Egyptian
Loaded as API: https://mohamedrashad-egyptian-arabic-tts.hf.space ✔
